In [1]:
# automatically update imports
%load_ext autoreload
%autoreload 2

# For graphs
%matplotlib inline

# snippet for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
import loaders

city_ref_dp = 'dublin_pleasanton'
bbox_dp = [37.6248, -121.9603, 37.7296, -121.8202]
pois_df_dp = loaders.load_pois(city_ref_dp, bbox_dp)
categories_kde_dict_dp = loaders.load_grid_kde(city_ref_dp, pois_df_dp, bbox_dp)

city_ref_g = 'girona'
bbox_g = [41.9262, 2.7692, 42.0237, 2.8489]
pois_df_g = loaders.load_pois(city_ref_g, bbox_g)
categories_kde_dict_g = loaders.load_grid_kde(city_ref_g, pois_df_g, bbox_g)

Saving data for `['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']` at `/home/martibosch/urbansprawl/urban_analysis/hdfs_store/girona.h5`
The data has been stored locally with success


Saving data for `['pois']` at `/home/martibosch/urbansprawl/urban_analysis/hdfs_store/girona.h5`
The data has been stored locally with success
Querying locally for `['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']`
`['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']` is/are not stored locally. Determining it/them through `get_all_kde_dict` method


Saving data for `['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']` at `/home/martibosch/urbansprawl/urban_analysis/hdfs_store/dublin_pleasanton.h5`
The data has been stored locally with success
Querying locally for `['pois']`
`['pois']` is/are not stored locally. Determining it/them through `pois_from_bbox` method


Querying locally for `['pois']`
Found ['pois'] stored locally
Querying locally for `['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']`
`['kde_shop', 'kde_residential', 'kde_commercial/industrial', 'kde_leisure/amenity', 'kde_global']` is/are not stored locally. Determining it/them through `get_all_kde_dict` method


In [26]:
import measures
import kde

xx_dp, yy_dp = kde._grid_from_bbox(bbox_dp, .0015)
xx_g, yy_g = kde._grid_from_bbox(bbox_g, .0015)

f_dp = categories_kde_dict_dp['kde_global'].values
f_g = categories_kde_dict_g['kde_global'].values

I_dp = measures.moran_index(xx_dp, yy_dp, f_dp)
I_g = measures.moran_index(xx_g, yy_g, f_g)

C_dp = measures.geary_index(xx_dp, yy_dp, f_dp)
C_g = measures.geary_index(xx_g, yy_g, f_g)

In [27]:
print('Dublin-Pleasanton: I = %f, C = %f' % (I_dp, C_dp))
print('Girona: I = %f, C = %f' % (I_g, C_g))

Dublin-Pleasanton: I = 0.203235, C = 0.847386
Girona: I = 0.113574, C = 1.041552


In [73]:
measures.relative_entropy(f_dp)

0.90235670672642254

In [74]:
measures.relative_entropy(f_g)

0.68305273991863913

In [16]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_categories_kde(categories_kde_dict, pois_df, xx, yy):
    n_rows = len(categories_kde_dict)
    base_figsize = 8

    fig = plt.figure(figsize=(8, 8 * n_rows))
    i = 1
    for label in ['amenity', 'shop', 'global']:
        f = categories_kde_dict[kde._CATEGORY_MAPPING[label]] # TODO: don't use the mapping!
        if label == 'global':
            category_pois = pois_df
        else:
            category_pois = pois_df[pois_df['key'] == label]
        ax1 = fig.add_subplot(n_rows, 2, i)
        ax2 = fig.add_subplot(n_rows, 2, i+1, projection='3d')
        ax1.scatter(category_pois['lon'], category_pois['lat'])
        ax1.set_ylabel(label, rotation=90)
        ax2.plot_surface(xx, yy, f, cmap=cm.RdYlBu)
        i += 2
        ax2.set_xlabel('lon', labelpad=-10)
        ax2.set_ylabel('lat', labelpad=-10)
        ax2.zaxis.set_rotate_label(False) # workaround
        ax2.set_zlabel(r'$f_c$', rotation=0, labelpad=-15)
        ax2.w_xaxis.set_ticklabels([])
        ax2.w_yaxis.set_ticklabels([])
        ax2.w_zaxis.set_ticklabels([])
    plt.show()

In [31]:
%matplotlib qt
plot_categories_kde(categories_kde_dict_dp, pois_df_dp, xx_dp, yy_dp)
plot_categories_kde(categories_kde_dict_g, pois_df_g, xx_g, yy_g)